Importando bibliotecas

In [2]:
from haversine import haversine
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import streamlit as st
import folium
from streamlit_folium import folium_static
import inflection

In [3]:
df = pd.read_csv(r'C:\Users\Leticia Furletti\Repos\FTC\project1\Datasets\zomato.csv')

Limpeza do data frame

Substituindo valores em branco (NaN) pelo valor mais utilizado na coluna

In [4]:
#print(df)
#print(df.dtypes)
#print(df.isna().sum())
#print(list(df["Cuisines"].unique()))
#print(df["Cuisines"].value_counts())
df["Cuisines"]=df["Cuisines"].fillna('Pizza, Fast Food')
#print(df.isna().sum())

Renomear colunas tirando espaço e coloca underline

In [5]:
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df
df = rename_columns(df)

Substituir codigos da coluna country_code por palavras

In [6]:
countries = { 
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}
def country_name(country_id):
    return countries[country_id]
df["country_code"]=df["country_code"].apply(country_name)

Substituir codigos da coluna price_range por palavras

In [7]:
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
df["price_range"]=df["price_range"].apply(create_price_tye)

Substituir codigos da coluna rating_colors por palavras

In [8]:
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]
df["rating_color"]=df["rating_color"].apply(color_name)

Coluna cuisines apresentava mais de um valor por celula, separando-os por virgula. Essa função mantera apenas o primeiro dos valores

In [9]:
df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

In [10]:
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes'],
      dtype='object')

Respondendo as perguntas - Geral

1. Quantos restaurantes únicos estão registrados?

In [10]:
len(df["restaurant_id"].unique())

6942

2. Quantos países únicos estão registrados?

In [22]:
len(df["country_code"].unique())

15

3. Quantas cidades únicas estão registradas?

In [20]:
len(df["city"].unique())

125

3.Qual o total de avaliações feitas?

In [14]:
df["votes"].sum()

4639654

In [16]:
len(df["cuisines"].unique())

165

Respondendo as perguntas - Pais

1. Qual o nome do país que possui mais cidades registradas?

In [80]:
df.loc[:,['city', 'country_code']].drop_duplicates().groupby('country_code').count().reset_index().sort_values('city', ascending = False).iloc[0,0]

'India'

 2. Qual o nome do país que possui mais restaurantes registrados?

In [79]:
df.loc[:,['restaurant_id', 'country_code']].drop_duplicates().groupby('country_code').count().reset_index().sort_values('restaurant_id', ascending = False).iloc[0,0]

'India'

3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [20]:
df.loc[df["aggregate_rating"] == 4,"country_code"].value_counts().reset_index().iloc[0,0]

'India'

In [62]:

nome_pais = list(df['country_code'].unique()) #vai me dar os paises existentes
total_paises =  0 #fazendo isso só pra definir minha variavel (que recebe integer)
nivel_preco = '' #fazendo isso só pra definir minha variavel (que recebe string)

for paises in nome_pais:
  filtro = (df['country_code'] == paises) & (df['aggregate_rating'] == 4  )
  resultante_do_filtro = df.loc[filtro, :].shape[0]
  if total_paises < resultante_do_filtro:
    total_paises  = resultante_do_filtro 
    nome_pais = paises

print(nome_pais)

India


4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [69]:
df.loc[:,['cuisines', 'country_code']].drop_duplicates().groupby('country_code').count().reset_index().sort_values('cuisines', ascending = False).iloc[0,0]

'India'

5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [62]:
df.loc[:,['aggregate_rating', 'country_code']].drop_duplicates().groupby('country_code').count().reset_index().sort_values('aggregate_rating', ascending = False).iloc[0,0]

'Brazil'

6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [104]:
df.loc[df["is_delivering_now"] > 0  ,"country_code"].value_counts().head(1)

India    1247
Name: country_code, dtype: int64

7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [150]:
df.loc[df["has_table_booking"] > 0  ,"country_code"].value_counts().reset_index().iloc[0,0]

'India'

8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [120]:
df.loc[:, ['aggregate_rating', 'country_code']].groupby('country_code').mean().sort_values('aggregate_rating', ascending = False).reset_index().iloc[0,0]

'Indonesia'

9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [24]:
aux=df.loc[:,['aggregate_rating', 'country_code']].groupby('country_code').mean().reset_index()
aux.loc[aux['aggregate_rating'] == aux['aggregate_rating'].max(), 'country_code']

5    Indonesia
Name: country_code, dtype: object

In [33]:
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,expensive,4.6,darkgreen,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,expensive,4.6,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,Botswana Pula(P),0,0,0,0,gourmet,4.7,darkgreen,Excellent,469
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,Botswana Pula(P),0,0,0,0,expensive,4.4,green,Very Good,867
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,Botswana Pula(P),0,0,0,0,expensive,4.4,green,Very Good,858


Qual o nome do país que possui, na média, a menor nota média registrada?

In [127]:
df.loc[:,['aggregate_rating', 'country_code']].groupby('country_code').min().reset_index().iloc[0,0]

'Australia'

11. Qual a média de preço de um prato para dois por país?

In [154]:
df.loc[:,['average_cost_for_two', 'country_code']].groupby('country_code').mean().sort_values('average_cost_for_two', ascending = False).reset_index()

,country_code,average_cost_for_two
0,Indonesia,312682.926829
1,Australia,138959.783333
2,Sri Lanka,2607.471264
3,Philippines,1211.090909
4,India,702.303964
5,South Africa,335.479058
6,Qatar,176.648936
7,United Arab Emirates,158.218563
8,Singapure,141.036585
9,Brazil,137.375479


Respondendo as perguntas - Cidade

1. Qual o nome da cidade que possui mais restaurantes registrados?

In [19]:
df.loc[:,['city', 'restaurant_id']].drop_duplicates().groupby('city').count().reset_index().sort_values('restaurant_id', ascending = False).iloc[0,0]

'Abu Dhabi'

2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4

In [126]:
df.loc[df["aggregate_rating"] >= 4, "city"].value_counts().reset_index().iloc[0,0]

'Kolkata'

In [125]:
(df.loc[df['aggregate_rating'] >= 4, ['restaurant_id', 'city'] ]
                                         .groupby('city')
                                         .count()
                                         .sort_values('restaurant_id', ascending=False)
                                         .reset_index()).iloc[0, 0]

'Kolkata'

3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [11]:
df.loc[df["aggregate_rating"] < 2.5, "city"].value_counts().reset_index().iloc[0,0]

'Gangtok'

4.Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [31]:
df.loc[:,['average_cost_for_two', 'city']].groupby('city').mean().reset_index().sort_values('average_cost_for_two', ascending = False).iloc[0,0]

'Adelaide'

5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [41]:
df.loc[:,['cuisines','city']].drop_duplicates().groupby('city').count().reset_index().sort_values('cuisines',ascending = False).iloc[0,0]

'Birmingham'

6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [66]:
df.loc[df["has_table_booking"] == 1  ,"city"].value_counts().head(1)

Bangalore    47
Name: city, dtype: int64

In [67]:
df.loc[df['has_table_booking'] == 1, ['restaurant_id', 'city'] ].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index().iloc[0, 0]

'Bangalore'

7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [70]:
df.loc[df['is_delivering_now'] == 1,['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending =False).reset_index().iloc[0,0]

'Aurangabad'

8. Qual o nome da cidade que possui a maior quantidade de restaurantes queaceitam pedidos online?

In [75]:
df.loc[df['has_online_delivery']==1,['restaurant_id','city']].groupby('city').count().reset_index().sort_values('restaurant_id',ascending =False).iloc[0,0]

'Abu Dhabi'

Respondendo as perguntas - Restaurantes

1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [105]:
df.loc[:,['restaurant_name', 'votes']].groupby('restaurant_name').count().reset_index().sort_values('votes', ascending=False).iloc[0,0]

"Domino's Pizza"

In [98]:
cols = [
    'restaurant_id', 'restaurant_name', 'country_code', 
    'city', 'cuisines', 'aggregate_rating', 'votes'
]

df.loc[:, cols].sort_values(['votes', 'restaurant_id'], ascending=[False, True]).iloc[0,1]

'Bawarchi'

2. Qual o nome do restaurante com a maior nota média?

In [114]:
df.loc[:,['restaurant_name','aggregate_rating']].groupby('restaurant_name').mean().reset_index().sort_values('aggregate_rating', ascending=False).iloc[0,0]

'Earl of Sandwich'

3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [121]:
#Domino's Pizza

,restaurant_name,average_cost_for_two
1528,Domino's Pizza,204
2605,KFC,51
3912,Pizza Hut,40
3226,McDonald's,36
458,Barbeque Nation,32
...,...,...
2088,Grillfish DC,1
2087,Grille Restaurant,1
2086,Grillaz,1
2085,Grill and Shakes,1
